* https://github.com/benoitc/gunicorn
* https://gunicorn.org/

In [ ]:
%%bash
sudo -HE pip install -r requirements.txt

In [ ]:
%%bash
gunicorn --reload --workers 4 myapp:app

* https://docs.python.org/3/library/logging.config.html#configuration-file-format
* https://github.com/benoitc/gunicorn/blob/master/examples/logging.conf

In [ ]:
%%writefile gunicorn-log.conf
[loggers]
keys=root, gunicorn.error

[handlers]
keys=error_console

[formatters]
keys=generic

[logger_root]
level=INFO
handlers=error_console

[logger_gunicorn.error]
level=INFO
handlers=error_console
propagate=0
qualname=gunicorn.error

[handler_error_console]
class=StreamHandler
formatter=generic
args=(sys.stderr, )

[formatter_generic]
format=%(asctime)s %(levelname)-5s [%(module)s] ~ %(message)s
datefmt=%Y-%m-%d %H:%M:%S %Z
class=logging.Formatter

* https://docs.gunicorn.org/en/stable/settings.html#logconfig

In [ ]:
%%bash
gunicorn --reload --workers 4 --log-config gunicorn-log.conf myapp:app

* https://docs.python.org/3/library/logging.config.html#logging.config.dictConfig

In [ ]:
# one example
{
  "version": 1,
  "disable_existing_loggers": true,
  "filters": {
    "skipDebug": {
      "()": "__main__.RemoveLevelFilter",
      "levelToSkip": "DEBUG"
    }
  },
  "formatters": {
    "simple": {
      "format": "%(asctime)s|%(name)s [%(levelname)s] - %(message)s"
    }
  },
  "handlers": {
    "console":{
      "level": "DEBUG",
      "class": "logging.StreamHandler",
      "formatter": "simple",
      "stream" : "ext://sys.stdout"
    },
    "file": {
      "level": "DEBUG",
      "class": "logging.handlers.RotatingFileHandler",
      "maxBytes": 5242880,
      "backupCount": 3,
      "formatter": "simple",
      "filename": "log.log",
      "mode": "a",
      "encoding": "utf-8",
      "filters": ["skipDebug"]
    }
  },
  "loggers": {},
  "root": {
    "handlers": ["console", "file"],
    "level": "DEBUG"
  }
}

In [ ]:
# another example
import sys, logging

class StdErrFilter(logging.Filter):
    def filter(self, rec):
        return rec.levelno in (logging.ERROR, logging.WARNING)

class StdOutFilter(logging.Filter):
    def filter(self, rec):
        return rec.levelno in (logging.DEBUG, logging.INFO)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(process)s - %(asctime)s - %(name)s - %(levelname)s - %(message)s')

h1 = logging.StreamHandler(sys.stdout)
h1.setLevel(logging.DEBUG)
h1.setFormatter(formatter)
h1.addFilter(StdOutFilter())
logger.addHandler(h1)

h2 = logging.StreamHandler(sys.stderr)
h2.setLevel(logging.WARNING)
h2.setFormatter(formatter)
h2.addFilter(StdErrFilter())
logger.addHandler(h2)

https://github.com/benoitc/gunicorn/blob/548d5828da6b93fa6a14217742c6e6d2c7b2b900/gunicorn/glogging.py#L48

In [ ]:
# original configurations
dict(
    version=1,
    disable_existing_loggers=False,

    root={"level": "INFO", "handlers": ["console"]},
    loggers={
        "gunicorn.error": {
            "level": "INFO",
            "handlers": ["error_console"],
            "propagate": True,
            "qualname": "gunicorn.error"
        },

        "gunicorn.access": {
            "level": "INFO",
            "handlers": ["console"],
            "propagate": True,
            "qualname": "gunicorn.access"
        }
    },
    handlers={
        "console": {
            "class": "logging.StreamHandler",
            "formatter": "generic",
            "stream": "ext://sys.stdout"
        },
        "error_console": {
            "class": "logging.StreamHandler",
            "formatter": "generic",
            "stream": "ext://sys.stderr"
        },
    },
    formatters={
        "generic": {
            "format": "%(asctime)s [%(process)d] [%(levelname)s] %(message)s",
            "datefmt": "[%Y-%m-%d %H:%M:%S %z]",
            "class": "logging.Formatter"
        }
    }
)

In [ ]:
# new configurations
{
    'version': 1,
    'filters': {
        'myfilter': {
            '()': MyFilter,
            'param': 'noshow',
        }
    },
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'filters': ['myfilter']
        }
    },
    'root': {
        'level': 'DEBUG',
        'handlers': ['console']
    },
}

* https://docs.gunicorn.org/en/stable/settings.html#logconfig-dict

In [ ]:
%%bash
gunicorn --reload --workers 4 --log-config-dict {} myapp:app

In [ ]:
%%bash
gunicorn --reload --workers 4 --log-config-dict '{"loggers":{"gunicorn.error":{"level":"ERROR"}}}' myapp:app 1> stdout.txt 2> stderr.txt

* https://docs.python.org/3/library/logging.config.html#configuration-file-format
* https://github.com/benoitc/gunicorn/blob/master/examples/logging.conf

In [ ]:
%%writefile gunicorn-log.conf
[loggers]
keys=root, gunicorn.error, gunicorn.access

[handlers]
keys=console, error_console

[formatters]
keys=generic, access

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=NOTSET
handlers=console, error_console
propagate=1
qualname=gunicorn.error

[logger_gunicorn.access]
level=INFO
handlers=console
propagate=0
qualname=gunicorn.access

[handler_console]
level=INFO
class=StreamHandler
formatter=generic
args=(sys.stdout, )

[handler_error_console]
level=ERROR
class=StreamHandler
formatter=generic
args=(sys.stderr, )

[formatter_generic]
format=%(asctime)s [%(process)d] [%(levelname)s] %(message)s
datefmt=%Y-%m-%d %H:%M:%S
class=logging.Formatter

[formatter_access]
format=%(message)s
class=logging.Formatter

* https://docs.gunicorn.org/en/stable/settings.html#logconfig

In [ ]:
%%bash
gunicorn --reload --workers 4 --bind 0.0.0.0:8888 --log-config gunicorn-log.conf myapp:app